## Chaine de traitement en Python pour adapter le moteur de reconnaissance d'entités nommées de Spacy par entrainement. 

Il s'agit de :


1.   annoter automatiquement en noms de lieux (LOC) et de personnes (PERS) un extrait du texte "Bel Ami" de Maupassant en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy ;
2.   transférer le texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations créées automatiquement;
3.   Corriger manuellement ces annotations en vue de créer un jeu  de données d'entrainement ;
4.   Formatter et intégrer les annotations corrigées dans Spacy en tant que jeu de données d'entrainement ;
5.  Initialiser un modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement ;
6.  Initialiser l'entrainement avec les paramètres définis dans le fichier .cfg et entrainement par le biais du CLI de Spacy ;
7.  Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant.





In [1]:
!git clone https://github.com/cvbrandoe/coursTAL.git

Cloning into 'coursTAL'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 135 (delta 39), reused 77 (delta 39), pack-reused 58
Receiving objects: 100% (135/135), 11.86 MiB | 20.25 MiB/s, done.
Resolving deltas: 100% (60/60), done.


**Prérequis : Installation ou mise à jour de Spacy et téléchargement du modèle**

In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.3 MB/s 
     |████████████████████████████████| 1.2 MB 38.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!python -m spacy info

2022-12-06 19:25:15.271001: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================== Info about spaCy ==============================

spaCy version    3.4.3                         
Location         /usr/local/lib/python3.8/dist-packages/spacy
Platform         Linux-5.10.133+-x86_64-with-glibc2.27
Python version   3.8.15                        
Pipelines        en_core_web_sm (3.4.1)        



In [4]:
!python -m spacy download fr_core_news_sm  

2022-12-06 19:25:26.183084: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 18.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


Plus de modèles Spacy entrainés sur : https://spacy.io/models 

**Chargement d'un extrait du premier chapiture de Bel Ami de Maupassant.**


**1. Annotation automatiquement en noms de lieux (LOC) et de personnes (PERS) du texte en entrée en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy**

In [5]:
import spacy
import json
import requests
import os

def get_class_id(label):
  """
  Translates the spaCy label id into the tagtog entity type id
  - label: spaCy label id
  """
  choices = {'PER': 'e_1', 'LOC': 'e_2', 'GPE': 'e_2'}
  return choices.get(label, None)

def get_entities(spans, pipeline):
  """
  Translates a tuple of named entity Span objects (https://spacy.io/api/span) into a 
  list of tagtog entities (https://docs.tagtog.net/anndoc.html#ann-json). Each entity is
  defined by the entity type ID (classId), the part name where the annotation is (part),
  the entity offsets and the confidence (annotation status, who created it and probabilty).
  - spans: the named entities in the spaCy doc
  - pipeline: trained pipeline name
  """
  default_prob = 1
  default_part_id = 's1v1'
  default_state = 'pre-added'
  tagtog_entities = []
  for span in spans:
    class_id = get_class_id(span.label_)
    if class_id is not None:
      tagtog_entities.append( {
        'classId': class_id,
        'part': default_part_id,
        'offsets':[{'start': span.start_char, 'text': span.text}],
        'confidence': {'state': default_state,'who': ['ml:' + pipeline],'prob': default_prob},
        'fields':{},
        # this is related to the kb_id (knowledge base ID) field from the Span spaCy object
        'normalizations': {}} )
  return tagtog_entities


In [6]:
def read_text(file_name):
  with open (file_name, "r", encoding="utf_8") as myfile:
    lines = list(line for line in (l.strip() for l in myfile) if line)
    return str(lines)

text = read_text("coursTAL/2022/belAmi_01-01.txt")

pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(text)

def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
            print("\n")             
show_ents(doc)


PREMIÈRE PARTIE - 2 - 17 - ORG - Companies, agencies, institutions, etc.


I - 22 - 23 - LOC - Non-GPE locations, mountain ranges, bodies of water


Quand la caissière - 27 - 45 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Georges Duroy - 97 - 110 - PER - Named person or family.


joli garçon - 366 - 377 - PER - Named person or family.


rue Notre-Dame-de-Lorette - 1386 - 1411 - LOC - Non-GPE locations, mountain ranges, bodies of water


Quoique habillé d’un complet - 1967 - 1995 - ORG - Companies, agencies, institutions, etc.


Paris - 2473 - 2478 - LOC - Non-GPE locations, mountain ranges, bodies of water


Georges Duroy - 2981 - 2994 - PER - Named person or family.


Champs-Élysées - 3112 - 3126 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


bois de Boulogne - 3142 - 3158 - LOC - Non-GPE locations, mountain ranges, bodies of water


Comment - 3281 - 3288 - LOC - Non-GPE locations, mountain ranges, bo

**2. transfer du texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations**

Au préalable, il faut créer un compte utilisateur et un projet dans TagTog (https://www.tagtog.com/) et modifier les informations de connexion ci-dessous.

Lors de la configuration (settings) de votre projet sur TagTog, dans la rubrique entities, il faut déclarer les trois catégories d'entités pour l'exercice (dans cet ordre là) :  **Personnages**, **Lieux** et **Misc** 

In [8]:
# Set the credentials at tagtog and project name
MY_USERNAME = 'cvbrandoe'
MY_PASSWORD = 'talEHESS'
MY_PROJECT = 'CoursENTAL'

# API authentication
tagtogAPIUrl = "https://www.tagtog.com/-api/documents/v1"
auth = requests.auth.HTTPBasicAuth(username=MY_USERNAME, password=MY_PASSWORD)

# Initialize ann.json (specification: https://docs.tagtog.net/anndoc.html#ann-json)
annjson = {}
# Set the document as not confirmed, an annotator will manually confirm whether the annotations are correct
annjson['anncomplete'] = False
annjson['metas'] = {}
annjson['relations'] = []                      
# Transform the spaCy entities into tagtog entities
annjson['entities'] = get_entities(doc.ents, pipeline)

print(text)
print(json.dumps(annjson))

# Parameters for the API call 
# see https://docs.tagtog.net/API_documents_v1.html#examples-import-pre-annotated-plain-text-file
params = {'owner': MY_USERNAME, 'project': MY_PROJECT, 'output': 'null', 'format': 'default-plus-annjson'}
# Pre-annotated document composed of the content and the annotations
files=[('BelAmi_extrait0101.txt', text), ('BelAmi_extrait0101.ann.json', json.dumps(annjson))]
# POST request to send the pre-annotated document
response = requests.post(tagtogAPIUrl, params=params, auth=auth, files=files)

print(response.text)

['PREMIÈRE PARTIE.', 'I', 'Quand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant.', 'Comme il portait beau, par nature et par pose d’ancien sous-officier, il cambra sa taille, frisa sa moustache d’un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s’étendent comme des coups d’épervier.', 'Les femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d’un chapeau toujours poussiéreux et vêtue d’une robe toujours de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.', 'Lorsqu’il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu’il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans dîners, au choi

**3.  Correction manuelle des annotations en vue de créer un jeu  de données d'entrainement**

Ensuite téléchargez le fichier json à partir de tagtog et renommez-le en "BelAmi_01_01_corrected.ann.json" et importez-le dans l'espace de fichiers, ce fichier est traité ci-dessous.

**4. Formattage et intégration des annotations corrigées dans Spacy en tant que jeu de données d'entrainement**

In [10]:
# importing the module
import json
  # reading the data from the file
with open('coursTAL/2022/BelAmi_01_01_corrected.ann.json') as f:
    data = f.read()  
print("Data type before reconstruction : ", type(data))
# reconstructing the data as a dictionary
js_corrected_training_data = json.loads(data)
print("Data type after reconstruction : ", type(js_corrected_training_data))
print(js_corrected_training_data)

Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'dict'>
{'anncomplete': False, 'metas': {}, 'relations': [], 'entities': [{'classId': 'e_1', 'part': 's1v1', 'offsets': [{'start': 95, 'text': 'Georges Duroy'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_2', 'part': 's1v1', 'offsets': [{'start': 1378, 'text': 'rue Notre-Dame-de-Lorette'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_2', 'part': 's1v1', 'offsets': [{'start': 2459, 'text': 'Paris'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_1', 'part': 's1v1', 'offsets': [{'start': 2963, 'text': 'Georges Duroy'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['m

In [11]:
def get_class_id_inv(label):
  """
  Translates the tagtog entity type id into spaCy label id 
  - label: tagtog entity type id
  """
  choices = {'e_1': 'PER', 'e_2': 'LOC', 'e_3': 'MISC'}
  return choices.get(label, None)

In [12]:


training_data = {'classes' : ['PER', "LOC", "MISC"], 'annotations' : []}
temp_dict = {}
temp_dict['text'] = text
temp_dict['entities'] = []
for example in js_corrected_training_data['entities']:  
  #print(example)
  start = example['offsets'][0]['start']
  end = int(example['offsets'][0]['start']) + len(example['offsets'][0]['text'])
  label = get_class_id_inv(example['classId'])
  temp_dict['entities'].append((start, end, label))
  #print(temp_dict['entities'])
training_data['annotations'].append(temp_dict)
  
print(training_data['annotations'][0])

{'text': "['PREMIÈRE PARTIE.', 'I', 'Quand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant.', 'Comme il portait beau, par nature et par pose d’ancien sous-officier, il cambra sa taille, frisa sa moustache d’un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s’étendent comme des coups d’épervier.', 'Les femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d’un chapeau toujours poussiéreux et vêtue d’une robe toujours de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.', 'Lorsqu’il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu’il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans dîner

**5.  Initialisation du modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement** 

In [13]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("fr") # load a new spacy model
doc_bin = DocBin() # create a DocBin object


In [14]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("coursTAL/2022/training_data.spacy") # save the docbin object


100%|██████████| 1/1 [00:00<00:00,  7.54it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


Le fichier de configuration base_config_nermodel.cfg est base_config_nermodel.cfg 

**6.  Initialisation de l'entrainement avec les paramètres définis (hyperparamètres) dans le fichier .cfg et entrainement par le biais du CLI de Spacy**

In [15]:
!python -m spacy init fill-config "coursTAL/2022/base_config_nermodel.cfg" "coursTAL/2022/config.cfg"

2022-12-06 19:38:54.040300: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
coursTAL/2022/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [17]:
!python -m spacy train "coursTAL/2022/config.cfg" --paths.train "coursTAL/2022/training_data.spacy" --paths.dev "coursTAL/2022/training_data.spacy" --output "coursTAL/2022/"


2022-12-06 19:41:01.607663: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: coursTAL/2022
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-06 19:41:08,298] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-06 19:41:08,320] [INFO] Pipeline: ['tok2vec', 'ner', 'parser']
INFO:spacy:Pipeline: ['tok2vec', 'ner', 'parser']
[2022-12-06 19:41:08,320] [INFO] Resuming training for: ['ner']
INFO:spacy:Resuming training for: ['ner']
[2022-12-06 19:41:08,334] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-06 19:41:08,335] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-06 19:41:09,491] [INFO] Initialized pipeline components: ['tok2vec', 'parser']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'parser']
✔ Initialized pipeline

=

**7. Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant**

In [ ]:
nlp_ner = spacy.load("model-best")

test_dataset = open("coursTAL/2022/belAmi_01-05.txt").read().replace("’","'")
doc = nlp_ner(test_dataset)

colors = {"PER": "#F67DE3", "LOC": "#7DF6D9", "MISC" : "#FFFFFF"}
options = {"colors": colors} 

show_ents(doc)

#spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

**A titre de comparaison, test avec ce même texte mais à partir du modèle de base fourni par Spacy, fr_core_news_sm.**

In [ ]:
pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(test_dataset)
show_ents(doc)

Remarque 1. lors de la mise à jour du modèle NER (idem pour d'autres couches comme POS) avec les annotations de l'utilisateur, un processus itératif se met en place afin de faire converger les prédictions faites (des poids) vers des valeurs optimales vis-à-vis des annotations de référence. A caque itération, on calcule le "gradient" qui correspond, en optimisation, aux dérivées partielles de la fonction de classification non linéaire recherchée (voir [1], vous trouverez aussi la notion de backpropagation clé dans ces approches de DL pour le TAL). Pour observer la manière dont la valeur du gradient évolue (et se stabilise) lors des itérations, il est conseillé de regarder le paramètre "losses" mis à jour par la méthode update(..., losses=losses).
[1]: https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1162/handouts/CS224N_DeepNLP_Week7_lecture3.pdf 
Aussi : https://spacy.io/usage/training

Remarque 2. Pour éviter le surapprentissage (overfitting) d'un modèle : vous pouvez identifier que votre modèle n'est pas bon lorsqu'il fonctionne bien sur les données d'entraînement mais ne donne pas de bons résultats sur des données nouvelles et non encore vues. Autrement dit, le modèle "mémorise" les données d'apprentissage et n'est pas performant avec les nouvelles données.